# Deep Learning: Convolutional Neural Networks

### Import Library

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/Users/naii/Desktop/project/cs265/env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/naii/Desktop/project/cs265/env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/naii/Desktop/project/cs265/env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/naii/Desktop/project/cs265/env

### กำหนดฟังก์ชันต่างๆรวมทั้งการสร้างโครงข่าย

In [5]:
def get_weights(shape):
    data = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(data)

def get_biases(shape):
    data = tf.constant(0.1, shape=shape)
    return tf.Variable(data)

def create_layer(shape):
    # Get the weights and biases 
    W = get_weights(shape)
    b = get_biases([shape[-1]])

    return W, b

def convolution_2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], 
            padding='SAME')

def max_pooling(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
            strides=[1, 2, 2, 1], padding='SAME')

In [6]:
# Get the MNIST data
mnist = input_data.read_data_sets('./mnist_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist_data/t10k-images-idx3-ubyte.gz
Extracting ./mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### กำหนดโครงข่าย CNN

In [9]:
# The images are 28x28, so create the input layer 
# with 784 neurons (28x28=784) 
x = tf.placeholder(tf.float32, [None, 784])

# Reshape 'x' into a 4D tensor 
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Define the first convolutional layer
W_conv1, b_conv1 = create_layer([5, 5, 1, 32])

# Convolve the image with weight tensor, add the 
# bias, and then apply the ReLU function
h_conv1 = tf.nn.relu(convolution_2d(x_image, W_conv1) + b_conv1)

# Apply the max pooling operator
h_pool1 = max_pooling(h_conv1)

# Define the second convolutional layer
W_conv2, b_conv2 = create_layer([5, 5, 32, 64])

# Convolve the output of previous layer with the 
# weight tensor, add the bias, and then apply 
# the ReLU function
h_conv2 = tf.nn.relu(convolution_2d(h_pool1, W_conv2) + b_conv2)

# Apply the max pooling operator
h_pool2 = max_pooling(h_conv2)

# Define the fully connected layer
W_fc1, b_fc1 = create_layer([7 * 7 * 64, 1024])

# Reshape the output of the previous layer
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# Multiply the output of previous layer by the 
# weight tensor, add the bias, and then apply 
# the ReLU function
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Define the dropout layer using a probability placeholder
# for all the neurons
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Define the readout layer (output layer)
W_fc2, b_fc2 = create_layer([1024, 10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### กำหนดการคำนวน loss และ gradient descent optimizer

In [10]:
# Define the entropy loss and the optimizer
y_loss = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_loss))
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### กำหนดการคำนวนประสิทธิภาพ

In [11]:
# Define the accuracy computation
predicted = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_loss, 1))
accuracy = tf.reduce_mean(tf.cast(predicted, tf.float32))

### สร้าง session ของ tensorflow

In [12]:
# Create and run a session
sess = tf.InteractiveSession()
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


## Train

In [9]:
# Start training
num_iterations = 21000
batch_size = 75
print('\nTraining the model....')
for i in range(num_iterations):
    # Get the next batch of images
    batch = mnist.train.next_batch(batch_size)

    # Print progress
    if i % 50 == 0:
        cur_accuracy = accuracy.eval(feed_dict = {
                x: batch[0], y_loss: batch[1], keep_prob: 1.0})
        print('Iteration', i, ', Accuracy =', cur_accuracy)

    # Train on the current batch
    optimizer.run(feed_dict = {x: batch[0], y_loss: batch[1], keep_prob: 0.5})


Training the model....
Iteration 0 , Accuracy = 0.10666667
Iteration 50 , Accuracy = 0.7733333
Iteration 100 , Accuracy = 0.84
Iteration 150 , Accuracy = 0.8933333
Iteration 200 , Accuracy = 0.8666667
Iteration 250 , Accuracy = 0.88
Iteration 300 , Accuracy = 0.93333334
Iteration 350 , Accuracy = 0.92
Iteration 400 , Accuracy = 0.94666666
Iteration 450 , Accuracy = 0.93333334
Iteration 500 , Accuracy = 0.93333334
Iteration 550 , Accuracy = 0.9066667
Iteration 600 , Accuracy = 0.93333334
Iteration 650 , Accuracy = 0.93333334
Iteration 700 , Accuracy = 0.94666666
Iteration 750 , Accuracy = 0.96
Iteration 800 , Accuracy = 0.96
Iteration 850 , Accuracy = 0.97333336
Iteration 900 , Accuracy = 0.97333336
Iteration 950 , Accuracy = 0.97333336
Iteration 1000 , Accuracy = 0.92
Iteration 1050 , Accuracy = 1.0
Iteration 1100 , Accuracy = 0.93333334
Iteration 1150 , Accuracy = 1.0
Iteration 1200 , Accuracy = 0.97333336
Iteration 1250 , Accuracy = 0.9866667
Iteration 1300 , Accuracy = 0.9866667
It

Iteration 11800 , Accuracy = 1.0
Iteration 11850 , Accuracy = 1.0
Iteration 11900 , Accuracy = 1.0
Iteration 11950 , Accuracy = 1.0
Iteration 12000 , Accuracy = 1.0
Iteration 12050 , Accuracy = 1.0
Iteration 12100 , Accuracy = 1.0
Iteration 12150 , Accuracy = 0.9866667
Iteration 12200 , Accuracy = 1.0
Iteration 12250 , Accuracy = 1.0
Iteration 12300 , Accuracy = 1.0
Iteration 12350 , Accuracy = 1.0
Iteration 12400 , Accuracy = 1.0
Iteration 12450 , Accuracy = 0.9866667
Iteration 12500 , Accuracy = 0.9866667
Iteration 12550 , Accuracy = 1.0
Iteration 12600 , Accuracy = 1.0
Iteration 12650 , Accuracy = 1.0
Iteration 12700 , Accuracy = 1.0
Iteration 12750 , Accuracy = 1.0
Iteration 12800 , Accuracy = 1.0
Iteration 12850 , Accuracy = 1.0
Iteration 12900 , Accuracy = 1.0
Iteration 12950 , Accuracy = 1.0
Iteration 13000 , Accuracy = 1.0
Iteration 13050 , Accuracy = 1.0
Iteration 13100 , Accuracy = 1.0
Iteration 13150 , Accuracy = 0.9866667
Iteration 13200 , Accuracy = 1.0
Iteration 13250 , A

## คำนวนประสิทธิภาพโดยใช้ชุดข้อมูลทดสอบ

In [10]:
# Compute accuracy using test data
print('Test accuracy =', accuracy.eval(feed_dict = {
        x: mnist.test.images, y_loss: mnist.test.labels, 
        keep_prob: 1.0}))

Test accuracy = 0.9907
